In [1]:
import pandas as pd
from itertools import combinations
import numpy as np
from tqdm import tqdm

**Clustering evaluation using Rand Index**

In [2]:
# evaluation function with an example
# the truths are the list of arxiv categories for each paper, the preds are the cluster labels
# rand score modified to allow for multiple categories per paper

def modified_rand_score_vectorized(preds, truths):
    n = len(preds)
    
    # Convert predictions to a numpy array for faster operations
    preds = np.array(preds)
    
    # Prepare a label presence matrix
    unique_labels = sorted(set(label for sublist in truths for label in sublist))
    label_index = {label: idx for idx, label in enumerate(unique_labels)}
    
    # Create a boolean matrix of size (n, number of unique labels)
    truth_matrix = np.zeros((n, len(unique_labels)), dtype=bool)
    for i, labels in enumerate(truths):
        truth_matrix[i, [label_index[label] for label in labels]] = True
    
    # Calculate pairwise matrix indicating shared label (the slowest part of the function))
    shared_label_matrix = np.dot(truth_matrix, truth_matrix.T) > 0
    
    # Calculate pairwise equal predictions
    same_pred = preds[:, None] == preds
    
    # Calculate TP, TN, FP, FN using vectorized operations
    TP = np.sum(np.logical_and(shared_label_matrix, same_pred))
    TN = np.sum(np.logical_and(~shared_label_matrix, ~same_pred))
    FP = np.sum(np.logical_and(~shared_label_matrix, same_pred))
    FN = np.sum(np.logical_and(shared_label_matrix, ~same_pred))

    # Correct for self-comparison (diagonal counts as True in both shared_label_matrix and same_pred)
    TP -= n
    
    # Calculate the Rand Index
    rand_index = (TP + TN) / (TP + TN + FP + FN) if (TP + TN + FP + FN) > 0 else 0
    return rand_index

# Example usage
preds = [1, 1, 2, 2]
truths = [['a', 'b'], ['a'], ['c'], ['c', 'd']]
print(modified_rand_score_vectorized(preds, truths))

1.0


**Example usage with arxiv data**

In [3]:
# function to add back the list of categories for each paper
def reorg_category_df(df_categories):
    data_ids, data_categories = [], []
    cur_id, cat = "", []
    data = df_categories.sort_values("id")

    for _, row in tqdm(data.iterrows(), total=data.shape[0]):
        if cur_id != row[0]:
            # save the exising id's categories data to dictionary. if cur_id is "", it indicates the beginning of the loop, no prior records to save
            if cur_id != "":
                data_ids.append(cur_id)
                data_categories.append(",".join(cat))
                cur_id = row[0]
                cat = []
            else:
                cur_id = row[0]
        cat.append(row[1])

    df = pd.DataFrame.from_dict({"id": data_ids, 'categories': data_categories})
    # save it to csv, so you could load it next time without rerunning
    # df.to_csv(save_to_file_name)
    # print(f"Total {df.shape[0]} records. Have saved the file {save_to_file_name} into the data folder.")

    return df

In [4]:
# the data/arxiv-metadata-ext-category.csv file contains N rows of the same paper (same paper id) with N different categories
df_categories = pd.read_csv("data/arxiv-metadata-ext-category.csv",dtype={"id":object,"category_id":object})
# Reorganize to each row as one paper with a column containing the list of categories. Might take 1 to 3+ mins.
df_reorg_category = reorg_category_df(df_categories)

100%|██████████| 4156055/4156055 [01:19<00:00, 52487.84it/s]


In [5]:
# load the clustering results with at least two columns: id and kmeans_label
df_clustering = pd.read_csv("filename.csv",dtype={"id":object})

In [6]:
df_clustering = df_clustering.merge(df_reorg_category, how='left', on="id")

In [7]:
def split_comma(input):
    try:
        return input.split(",")
    except:
        return []

In [8]:
df_clustering["categories_list"] = df_clustering["categories"].apply(lambda x: split_comma(x))

In [9]:
output_score = []
# to speed up the evaluation, we sample random 10,000 papers 10 times and evaluate the clustering results based on the average score
for i in range(10):
    df_clustering_filter = df_clustering[~df_clustering["categories"].isna()]
    df_sample = df_clustering_filter.sample(10000, random_state = i)
    score = modified_rand_score_vectorized(df_sample["kmeans_label"], df_sample["categories_list"])
    output_score.append(score)
    print(f"seed: {i}, score: {score}")
    
print(f"Average Rand Index Score: {np.mean(output_score)}")

seed: 0, score: 0.7954620262026203
seed: 1, score: 0.7983003700370037
seed: 2, score: 0.7919541354135413
seed: 3, score: 0.7935656565656566
seed: 4, score: 0.7985075707570757
seed: 5, score: 0.7984138613861386
seed: 6, score: 0.7968819681968197
seed: 7, score: 0.7975040504050405
seed: 8, score: 0.7983177117711772
seed: 9, score: 0.7969073307330733
Average Rand Index Score


0.7965814681468146

**Clustering evaluation using Silhouette Score (if possible)**

In [ ]:
# X is a feature array and labels are predicted labels for each sample
# reference: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.silhouette_score.html
silhouette_score(X, labels)